In [21]:
def counter_filter(raw_dataset, count):
    import collections
    counter = collections.Counter([tk for tk in raw_dataset])
    counter = dict(filter(lambda x: x[1] >= count, counter.items()))
    return counter

counter_filter([1,1,2],1)

{1: 2, 2: 1}

In [1]:
import json

import yaml
from tqdm import tqdm

# from config import get_config

# cfg = get_config('config/kkbox.yml')

def add_l(entity_dict, item):
    entity_dict[item] = True

############################ load raw-data ##############################

# song_id,song_length,genre_ids,artist_names,composers,lyricist,language
songs = {}
genre_ids = {} # 2
artist_names = {}
composers = {}
# lyricist = {}
# language = {}
kg_triples = set()
with open('data/kkbox/songs.csv', 'r', encoding='utf-8') as f:
    is_title = True
    for line in tqdm(f, total=2300000):
        if is_title:
            is_title = False
        else:
            info = line.split(',')
            add_l(songs, info[0])
            add_l(genre_ids, info[2])
            add_l(artist_names, info[3])
#             add_l(composers, info[4])
            kg_triples.add((info[0], info[2], info[3], info[4]))
            # add_l(lyricist, info[5])
            # add_l(language, info[6])

users = {}
with open('data/kkbox/members.csv', 'r', encoding='utf-8') as f:
    is_title = True
    for line in tqdm(f, total=34405):
        if is_title:
            is_title = False
        else:
            info = line.split(',')
            add_l(users, info[0])



100%|█████████▉| 7377419/7377420 [00:14<00:00, 497967.22it/s]


In [ ]:
# msno,song_id,source_system_tab,source_screen_name,source_type,target
interactions = set()
with open('data/kkbox/train.csv', 'r', encoding='utf-8') as f:
    is_title = True
    for line in tqdm(f, total=7377420):
        if is_title:
            is_title = False
        else:
            info = line.strip().split(',')
            interactions.add((info[0], info[1], info[-1]))


In [63]:
users_list = [u for u,i,_ in interactions]
items_list = [i for u,i,_ in interactions]
len(items_list)

7377418

In [64]:
users_set = counter_filter(users_list, 200)
items_set = counter_filter(items_list, 200)
users_list = [u for u,i,s in interactions if u in users_set and i in items_set]
items_list = [i for u,i,_ in interactions if u in users_set and i in items_set]
len(items_list)

3837594

In [14]:
items_list[0]

'3uNgEeV2o+wAhBhPjWNhgE8QoLEysbq5VhufbEZf6nc='

In [65]:
id2entity = {}
entity2id = {}
id2type = {}
id = 0

genre_set = set([t[1] for t in kg_triples if t[0] in items_set])
artist_set = set([t[2] for t in kg_triples if t[0] in items_set])

for user in tqdm(users_set, total=len(users_set)):
    id2entity[id] = user
    entity2id[user] = id
    id2type[id] = 'u'
    id += 1

c = 0
for entity in tqdm(items_set, total=len(items_set)):
#     if entity in items_set.keys():
        id2entity[id] = entity
        entity2id[entity] = id
        id2type[id] = 's'
        id += 1
        c += 1

for entity in tqdm(genre_set, total=len(genre_set)):
    id2entity[id] = entity
    entity2id[entity] = id
    id2type[id] = 'g'
    id += 1

for entity in tqdm(artist_set, total=len(artist_set)):
    id2entity[id] = entity
    entity2id[entity] = id
    id2type[id] = 'a'
    id += 1

100%|██████████| 1274/1274 [00:00<00:00, 570282.10it/s]


In [66]:
id

19118

In [6]:

# deprecated
#
# id2entity = {}
# entity2id = {}
# id2type = {}
# id = 0
# for user in users.keys():
#     id2entity[id] = user
#     entity2id[user] = id
#     id2type[id] = 'u'
#     id += 1

# for entity in songs.keys():
#     id2entity[id] = entity
#     entity2id[entity] = id
#     id2type[id] = 's'
#     id += 1

# for entity in genre_ids.keys():
#     id2entity[id] = entity
#     entity2id[entity] = id
#     id2type[id] = 'g'
#     id += 1

# for entity in artist_names.keys():
#     id2entity[id] = entity
#     entity2id[entity] = id
#     id2type[id] = 'a'
#     id += 1

# # for entity in composers.keys():
# #     id2entity[id] = entity
# #     entity2id[entity] = id
# #     id2type[id] = 'c'
# #     id += 1




In [69]:
############################ save dataset ############################


with open('data/kkbox/filtered/kg_triples.txt', 'w', encoding='utf-8') as f:
    for kg_triple in tqdm(kg_triples, total=len(kg_triples)):
        try:
            song = str(entity2id[kg_triple[0]])
            genre_id = str(entity2id[kg_triple[1]])
            artist = str(entity2id[kg_triple[2]])
            f.write(song+'\t'+'g'+'\t'+genre_id+'\n')
            f.write(song+'\t'+'a'+'\t'+artist+'\n')
        except:
            pass


# save interactions():
with open('data/kkbox/filtered/interactions.txt', 'w', encoding='utf-8') as f:
    for triple in tqdm(interactions, total=len(interactions)):
#         if triple[0] in users_set and triple[1] in items_set and int(float(triple[2]))==1:
        try:
            user = str(entity2id[triple[0]])
            item = str(entity2id[triple[1]])
            score = str(int(float(triple[2])))
            f.write(user+'\t'+item+'\t'+score+'\n')
        except:
            pass
            
# save_dict():
with open('data/kkbox/filtered/id2entity.dict', 'w', encoding='utf-8') as f:
    f.write(json.dumps(id2entity))
with open('data/kkbox/filtered/entity2id.dict', 'w', encoding='utf-8') as f:
    f.write(json.dumps(entity2id))
with open('data/kkbox/filtered/id2type.dict', 'w', encoding='utf-8') as f:
    f.write(json.dumps(id2type))



# load_dict()
# with open('data/kkbox/id2entity.dict', 'r', encoding='utf-8') as f:
#     id2entity = json.loads(f.read())
# with open('data/kkbox/entity2id.dict', 'r', encoding='utf-8') as f:
#     entity2id = json.loads(f.read())
# with open('data/kkbox/id2type.dict', 'r', encoding='utf-8') as f:
#     id2type = json.loads(f.read())


100%|██████████| 7377418/7377418 [00:14<00:00, 512798.19it/s]
